# Trabajo del fin de Semana

In [10]:
#imports necesarios
from bs4 import BeautifulSoup
import pandas as pd
import requests
import plotly.graph_objs as go
import plotly

In [2]:
#Definimos un método que obtenga los resultados para cada set de artículos que le enviemos
def ObtenerResultados(articulos, nombre, precio, precioOriginal, precioMensualidades):
    #Hacemos un ciclo para ir obteniendo los valores para cada uno de los arreglos
    for articulo in articulos :
        nombre.append(articulo.find('span', class_='main-title').get_text().strip())

        #Validamos si existe Precio, sino, ingresamos 0
        priceFraction = articulo.find('span', class_='price__fraction')
        if(priceFraction != None) :
            precio.append(priceFraction.get_text().strip())
        else :
            precio.append('0')

        #Validamos si existe Precio Original, sino, ingresamos 0
        priceOld = articulo.find('span', class_='price-old')
        if(priceOld != None) : 
            precioOriginal.append(priceOld.get_text().strip())
        else :
            precioOriginal.append("NA")

        #Validamos si existe Mensualidades, sino, ingresamos 0
        mensualidades = articulo.find('span', class_='item-installments-price')
        if(mensualidades != None) : 
            precioMensualidades.append(mensualidades.get_text().strip())
        else :
            precioMensualidades.append('0')

In [3]:
#Método que manda obtener el código de la página que queremos y lo transforma a lxml para poder navegarlo
def ObtenerURL(URLname) :
    URL = BeautifulSoup(requests.get(URLname).text, "lxml")
    return URL

#Método que regresa el listado de artículos de una página de MercadoLibre
def ObtenerArticulos(URL) :
    return URL.find_all('div', class_='item__info-container')

#Limpiamos los strings de precio quitando caracteres raros y espacios
def LimpiarPrecios(precios) :
    return [p.replace(u'\xa0', ' ').replace('$ ', '').replace(',', '').replace(' ', '.') for p in precios]

def ConvertirFloat(precios) :
    return [float(p) for p in precios]

In [4]:
#URL principal
URLname = 'https://electronica.mercadolibre.com.mx/televisores/#menu=categories'


In [5]:
#Obtenemos el listado completo de artículos
URL = ObtenerURL(URLname)

articulos = ObtenerArticulos(URL)

#Creamos listas vacías que vamos a llenar con la información del listado de artículos
nombre = []
precio = []
precioOriginal = []
precioMensualidades = []

#Llenamos las listas con información de artículos
ObtenerResultados(articulos, nombre, precio, precioOriginal, precioMensualidades)

#Limpiamos y convertimos el precio
precio = LimpiarPrecios(precio)
precio = ConvertirFloat(precio)

#Limpiamos el precio Original
precioOriginal = LimpiarPrecios(precioOriginal)

#Limpiamos y convertimos el precio de mensualidades
precioMensualidades = LimpiarPrecios(precioMensualidades)
precioMensualidades = ConvertirFloat(precioMensualidades)

In [6]:
#Generamos el dataframe principal
df = pd.DataFrame({'Nombre': nombre, 'Precio': precio, 'Precio Original' : precioOriginal, 'Precio 12 Meses' : precioMensualidades})

#Creamos la columna de dólares
dolares = df['Precio']/20

#Agregamos la nueva columna al dataframe
df['Precio en Dólares'] = dolares

#Guardamos el archivo csv
df.to_csv('Dataframe Precios.csv')

In [7]:
#Obtenemos los url de las siguientes páginas
paginasURL = URL.find_all('a', class_='andes-pagination__link')

#Recorremos las páginas a partir de la 2
for x in range(1, 3) :
    URLname = paginasURL[x]['href']
  
    #Obtenemos el listado completo de artículos para la página
    URL = ObtenerURL(URLname)

    articulos = ObtenerArticulos(URL)
    
    #Creamos listas vacías que vamos a llenar con la información del listado de artículos
    nombre = []
    precio = []
    precioOriginal = []
    precioMensualidades = []
    
    #Llenamos las listas con información de artículos
    ObtenerResultados(articulos, nombre, precio, precioOriginal, precioMensualidades)
    
    #Limpiamos y convertimos el precio
    precio = LimpiarPrecios(precio)
    precio = ConvertirFloat(precio)

    #Limpiamos el precio Original
    precioOriginal = LimpiarPrecios(precioOriginal)

    #Limpiamos y convertimos el precio de mensualidades
    precioMensualidades = LimpiarPrecios(precioMensualidades)
    precioMensualidades = ConvertirFloat(precioMensualidades)
    
    #Generamos un nuevo dataframe
    dfNuevo = pd.DataFrame({'Nombre': nombre, 'Precio': precio, 'Precio Original' : precioOriginal, 'Precio 12 Meses' : precioMensualidades})
    
    #Creamos la columna de dólares
    dolares = dfNuevo['Precio']/20

    #Agregamos la nueva columna al dataframe
    dfNuevo['Precio en Dólares'] = dolares
    
    df = pd.concat([df, dfNuevo])
    
df.to_csv('Dataframe Precios Completos.csv')


In [13]:
#Histograma
fig = go.Figure(data=[go.Histogram(x = df['Precio'])])
fig.show()